## Feature engineering

In [1]:
# importing libraries
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
churn_eng = pd.read_csv("../data/df_from_eda.csv")

In [3]:
churn_eng.head(3)

,player_id,date,gender,vertical,deposit_approved_sum,withdrawal_approved_sum,turnover_cash_sum,winning_cash_sum,result_cash_sum,NGR_sum,bonus_cash_sum,session_sum
0,0004fd6b7433920d183e818e18efb97c1b02afb3,2013-01-21,female,NaN,31,0,0,0,0,0,0,2233
1,0004fd6b7433920d183e818e18efb97c1b02afb3,2013-01-22,female,NaN,0,0,0,0,0,0,0,1957
2,0004fd6b7433920d183e818e18efb97c1b02afb3,2013-01-29,female,casino_classic,0,0,73,43,-30,30,0,2313


### Convert `date`column to datetime format

In [4]:
# convert object column to datetime format
churn_eng["date"] = pd.to_datetime(churn_eng["date"])

Create 4 new columns from `date`:
* `day_of_the_week`
* `day_of_the_month`
* `month`
* `week_of_year`

In [5]:
# create new features from date
# churn_eng["day_of_week"] = churn_eng["date"].dt.weekday  # day of the week (0 = Monday)
# churn_eng["day_of_month"] = churn_eng["date"].dt.day  # day of the month
# churn_eng["month"] = churn_eng["date"].dt.month  # month
# churn_eng["week_of_year"] = churn_eng["date"].dt.isocalendar().week  # week of the year

The format of the feature `player_id` is not useful neither practical. So it will be modified.

In [6]:
# convert player_id to category to label enconder it
churn_eng["player_id"] = churn_eng["player_id"].astype("category")
churn_eng["player_id_cat"] = churn_eng["player_id"].cat.codes
churn_eng["player_id_cat"] = churn_eng["player_id_cat"] + 1

### Removing features

We can remove `player_id` now.

In [7]:
# removing features
# churn_eng = churn_eng.drop(["player_id", "date"], axis=1)
churn_eng = churn_eng.drop(["player_id"], axis=1)

Since the missing values are less than 1% for `gender` and about 3% for `vertical` they will be removed.

In [8]:
# it will be used in a naive definition of the target variable
churn_naive = churn_eng.copy().reset_index(drop=True)

#### Removing missing values

In [9]:
# remove row with missing value for gender and vertical
churn_eng.dropna(axis=0, subset=["gender"], inplace=True)
churn_eng.dropna(axis=0, subset=["vertical"], inplace=True)

In [10]:
# total number of missing values, if they are present
print("Features with missing values:\n")
churn_eng.isnull().sum()[churn_eng.isnull().sum() > 0]

Features with missing values:



Series([], dtype: int64)

No more missing values!

In [11]:
# showing new dimensions of the dataset
print("Dimensions of the dataset:")
print(f" Number of rows: {churn_eng.shape[0]}")
print(f" Number of columns: {churn_eng.shape[1]}")
print()

Dimensions of the dataset:
 Number of rows: 112851
 Number of columns: 12



### Reordering columns

For the sake of clarity, the order of the columns in the dataframe will be changed.

In [12]:
# actual order of columns
list(churn_eng.columns)

['date',
 'gender',
 'vertical',
 'deposit_approved_sum',
 'withdrawal_approved_sum',
 'turnover_cash_sum',
 'winning_cash_sum',
 'result_cash_sum',
 'NGR_sum',
 'bonus_cash_sum',
 'session_sum',
 'player_id_cat']

In [13]:
# creating a list with the new order for the columns
cols_new_order = [
    "player_id_cat",
    "gender",
    "vertical",
    "date",
    #    "day_of_week",
    #    "day_of_month",
    #    "month",
    #    "week_of_year",
    "deposit_approved_sum",
    "withdrawal_approved_sum",
    "turnover_cash_sum",
    "winning_cash_sum",
    "result_cash_sum",
    "NGR_sum",
    "bonus_cash_sum",
    "session_sum",
]

In [14]:
# reordereing columns
churn_eng = churn_eng.reindex(columns=cols_new_order)

In [15]:
churn_eng.head(3)

,player_id_cat,gender,vertical,date,deposit_approved_sum,withdrawal_approved_sum,turnover_cash_sum,winning_cash_sum,result_cash_sum,NGR_sum,bonus_cash_sum,session_sum
2,1,female,casino_classic,2013-01-29,0,0,73,43,-30,30,0,2313
3,1,female,casino_classic,2013-02-07,0,0,0,0,0,0,0,6007
4,1,female,casino_classic,2013-02-18,0,0,0,0,0,0,0,3601


In [16]:
# plt.scatter(churn_eng["player_id_cat"], churn_eng["session_sum"])
# plt.show()
# plt.close()

In [17]:
# plt.scatter(churn_eng["player_id_cat"], churn_eng["result_cash_sum"])
# plt.show()
# plt.close()

In [18]:
# plt.scatter(churn_eng["player_id_cat"], churn_eng["deposit_approved_sum"])
# plt.show()
# plt.close()

In [19]:
# plt.scatter(churn_eng["player_id_cat"], churn_eng["NGR_sum"])
# plt.show()
# plt.close()

In [20]:
# save dataframe to file
churn_eng.to_csv("../data/df_from_eng.csv", index=False)

### Selecting only `casino_classic` gamblers for the model

In [21]:
# groups of gamblers in %
churn_eng.vertical.value_counts(normalize=True)

casino_classic    0.712665
sports            0.157960
casino_live       0.101576
mixture           0.027798
Name: vertical, dtype: float64

From the four groups of gamblers contained in the dataset the `casino_classic` counts for 71.3%. Considering this and the fact that different groups of gamblers can have different behavior, we will only model here the `casino_classic` group.

In [22]:
# make a copy of the complere dataset
casino_class = churn_eng.copy().reset_index(drop=True)

# filter only `casino_classic` gamblers
casino_class = casino_class[casino_class["vertical"] == "casino_classic"]

# drop vertical column (all dataset is casino_classic now)
casino_class = casino_class.drop(["vertical"], axis=1)

In [23]:
# comparing dimensions between the complete and the filtered dataset
churn_eng.shape, casino_class.shape

((112851, 12), (80425, 11))

In [24]:
casino_class.head()

,player_id_cat,gender,date,deposit_approved_sum,withdrawal_approved_sum,turnover_cash_sum,winning_cash_sum,result_cash_sum,NGR_sum,bonus_cash_sum,session_sum
0,1,female,2013-01-29,0,0,73,43,-30,30,0,2313
1,1,female,2013-02-07,0,0,0,0,0,0,0,6007
2,1,female,2013-02-18,0,0,0,0,0,0,0,3601
3,1,female,2013-03-06,0,0,0,0,0,0,0,2099
4,1,female,2013-03-07,0,0,0,0,0,0,0,4256


In [25]:
# save dataframe to file
casino_class.to_csv("../data/df_from_casino_classic.csv", index=False)

### Grouping by player_id and gender

Since each player can be represented by more than one data point we'll group them.

In [26]:
# columns
cols = [
    "deposit_approved_sum",
    "withdrawal_approved_sum",
    "turnover_cash_sum",
    "winning_cash_sum",
    "result_cash_sum",
    "NGR_sum",
    "bonus_cash_sum",
    "session_sum",
]

In [27]:
# group by player_id and gender
casino_class_grp = (
    casino_class.groupby(["player_id_cat", "gender"])[cols].sum().reset_index()
)

In [28]:
casino_class_grp.head()

,player_id_cat,gender,deposit_approved_sum,withdrawal_approved_sum,turnover_cash_sum,winning_cash_sum,result_cash_sum,NGR_sum,bonus_cash_sum,session_sum
0,1,female,87,146,309,337,28,-29,106,81819
1,2,male,1915,109,5003,3197,-1806,1785,98,28623
2,4,male,91,0,280,189,-91,90,0,5468
3,8,female,59,0,103,45,-58,59,41,2122
4,9,male,1885,1345,8751,8036,-715,513,69,23072


### One-hot enconding gender

In [29]:
# create the encoder
onehot = OneHotEncoder()

# encode the data
onehot_results = onehot.fit_transform(casino_class_grp[["gender"]])

# transform to pandas and join dataframes
casino_class_grp = casino_class_grp.join(
    pd.DataFrame(
        onehot_results.toarray(), columns=["female", "male"]
    )  # female=1, male=2
)

# drop gender column
casino_class_grp = casino_class_grp.drop(["gender"], axis=1)

In [30]:
# show dimensions
casino_class_grp.shape

(6808, 11)

In [31]:
# save dataframe to file
casino_class_grp.to_csv("../data/df_from_casino_class_grp.csv", index=False)

### Naive definition of the target variable

Steps:
* calculate the time difference in days between two sucessive events for each unique player
* keep only the last event containing the number of days for each player
* define an interval of time (time_lapse) below which we'll consider as `no_churn`

In [32]:
# calculate time difference between two sucessive dates for each unique identifier (player_id)
churn_naive["time_lapse_days"] = (
    churn_naive.groupby("player_id_cat")["date"]
    .diff()
    .apply(lambda x: x / np.timedelta64(1, "D"))
    .fillna(0)
    .astype("int64")
)

In [33]:
# keep only the last event for each unique ientifier (player_id)
churn_naive.drop_duplicates(subset="player_id_cat", keep="last", inplace=True)

In [34]:
# remove row with missing value for gender and vertical
churn_naive.dropna(axis=0, subset=["vertical"], inplace=True)
churn_naive.dropna(axis=0, subset=["gender"], inplace=True)

In [35]:
# make a copy of the complere dataset
casino_class_naive = churn_naive.copy().reset_index(drop=True)

# filter only `casino_classic` gamblers
casino_class_naive = casino_class_naive[
    casino_class_naive["vertical"] == "casino_classic"
]

# drop vertical column (all dataset is casino_classic now)
casino_class_naive = casino_class_naive.drop(["vertical"], axis=1)

In [36]:
# set the the time lapse in days below which we consider as a no_churn
time_lapse = 10
casino_class_naive["no_churn"] = np.where(
    casino_class_naive["time_lapse_days"] < time_lapse, 1, 0
)

In [37]:
# percentage of positive and negative class
casino_class_naive["no_churn"].value_counts(normalize=True)

1    0.746673
0    0.253327
Name: no_churn, dtype: float64

In [38]:
# one-hot encoder the gender
casino_class_naive = pd.get_dummies(casino_class_naive, columns=["gender"])

In [39]:
# visualize time lapse distribution
#plt.hist(casino_class_naive.time_lapse_days)
#plt.show()
#plt.close()

In [40]:
# visualize time lapse
#plt.figure(figsize=(14,10))
#plt.bar(casino_class_naive.player_id_cat, casino_class_naive.time_lapse_days)
#plt.show()
#plt.close()

In [41]:
# visualize player_id versus time lapse
#plt.scatter(casino_class_naive["player_id_cat"], casino_class_naive["time_lapse_days"])
#plt.show()
#plt.close()

In [42]:
# drop features
casino_class_naive = casino_class_naive.drop(["date"], axis=1)
casino_class_naive = casino_class_naive.drop(["time_lapse_days"], axis=1)

In [43]:
cols_new_order = [
    "player_id_cat",
    "deposit_approved_sum",
    "withdrawal_approved_sum",
    "turnover_cash_sum",
    "winning_cash_sum",
    "result_cash_sum",
    "NGR_sum",
    "bonus_cash_sum",
    "session_sum",
    "gender_female",
    "gender_male",
    "no_churn"
]

# reordereing columns
casino_class_naive = casino_class_naive.reindex(columns=cols_new_order)

In [44]:
casino_class_naive.head()

,player_id_cat,deposit_approved_sum,withdrawal_approved_sum,turnover_cash_sum,winning_cash_sum,result_cash_sum,NGR_sum,bonus_cash_sum,session_sum,gender_female,gender_male,no_churn
0,1,0,0,0,0,0,0,27,1876,1,0,1
1,2,90,0,103,13,-90,90,0,463,0,1,0
2,4,91,0,280,189,-91,90,0,5468,0,1,1
3,8,59,0,103,45,-58,59,41,2122,1,0,1
4,9,351,527,1234,1410,176,-186,0,3396,0,1,1


In [45]:
# save dataframe to file
casino_class_naive.to_csv("../data/train_naive.csv", index=False)